In [1]:
import numpy as np
import pandas as pd

In [2]:
import pulp

In [3]:
import tikzplotly

In [4]:
from src.modeling import *

In [5]:
from src.graphs import *

In [6]:
def of_net_income(_cost, _vars, _crops_year):
    return pulp.lpSum([getattr(_cost, _crops_year[i]) * _vars[i] for i in _crops_year])

def of_ua_total(_vars, _crops_year):
    return pulp.lpSum([_vars[i] for i in _crops_year])

def of_crop_diversity(_vars, _crops_year):
    return pulp.lpSum([_vars[i] for i in _crops_year])

In [7]:
sig = lambda sigma, NUM_ITER: 1/(1 + np.exp(sigma*(np.linspace(0, NUM_ITER, NUM_ITER)-(NUM_ITER/2))))

alpha_lin = np.linspace(0, 1, 1000)
alpha_sig_004 = sig(-0.004, 1000)
alpha_sig_06 = sig(-0.06, 1000)

l_alpha = {'alpha_lin':     alpha_lin, 
           #'alpha_sig_004': alpha_sig_004, 
           #'alpha_sig_06':  alpha_sig_06,
           }

In [8]:
variables = pd.read_csv('dados/al_rasheed/variables.csv', sep=';')
income = pd.read_csv('dados/al_rasheed/net_income.csv', sep=';')
constraints = pd.read_csv('dados/al_rasheed/constraints.csv', sep=';')
rotations = pd.read_csv('dados/al_rasheed/rotations.csv', sep=';').fillna(0)

In [9]:
YEARS = 2

In [10]:
crops = variables.variable.to_list()

crops_year_integer = dict(zip([f"y{i:03}_int_{j}" for i in range(1,YEARS+1) for j in crops], YEARS*crops))

crops_year_binary = dict(zip([f"y{i:03}_bin_{j}" for i in range(1,YEARS+1) for j in crops], YEARS*crops))

In [11]:
variables.set_index(keys='variable', inplace=True)

integer_vars = create_integer_vars(crops_year_integer, variables)

binary_vars = create_binary_vars(crops_year_binary, variables)

In [12]:
ofs = [of_net_income(income.loc[0,:], integer_vars, crops_year_integer),
       of_ua_total(integer_vars, crops_year_integer),
       of_crop_diversity(binary_vars, crops_year_binary)]

In [13]:
min_max_ofs = []
for of in ofs:
    vl = []
    for opt in [pulp.LpMinimize, pulp.LpMaximize]:
        model = pulp.LpProblem("Modelo_1", opt)
        model += of
        const_idx = add_constraints(model, integer_vars, constraints, crops_year_integer, len(crops), YEARS, const_id = 0)
        const_idx = add_binary_correlation(model, binary_vars, integer_vars, 113, const_id = const_idx)
        const_idx = add_crop_rotation_constraints(model, integer_vars, crops_year_integer, rotations, YEARS, crops, 'y{:03d}_int_{}', const_idx)

        solution = model.solve(pulp.GLPK())

        vl.append(pulp.value(model.objective))
    
    min_max_ofs.append(vl)

GLPSOL--GLPK LP/MIP Solver 5.0
Parameter(s) specified in the command line:
 --cpxlp /tmp/ac4e0e2e31f8454fb08ea4e96da45899-pulp.lp -o /tmp/ac4e0e2e31f8454fb08ea4e96da45899-pulp.sol
Reading problem data from '/tmp/ac4e0e2e31f8454fb08ea4e96da45899-pulp.lp'...
88 rows, 48 columns, 430 non-zeros
48 integer variables, 24 of which are binary
311 lines were read
GLPK Integer Optimizer 5.0
88 rows, 48 columns, 430 non-zeros
48 integer variables, 24 of which are binary
Preprocessing...
24 constraint coefficient(s) were reduced
64 rows, 48 columns, 242 non-zeros
48 integer variables, 24 of which are binary
Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  2.260e+03  ratio =  2.260e+03
GM: min|aij| =  4.453e-01  max|aij| =  2.246e+00  ratio =  5.042e+00
EQ: min|aij| =  2.100e-01  max|aij| =  1.000e+00  ratio =  4.761e+00
2N: min|aij| =  1.104e-01  max|aij| =  1.562e+00  ratio =  1.416e+01
Constructing initial basis...
Size of triangular part is 64
Solving LP relaxation...
GLPK Simplex Optimizer 5.

In [14]:
ofs_norn = [ min_max_normalization(ofs[idx], min_max_ofs[idx]) for idx in range(len(ofs))]

In [15]:
solution_lst = []
for key in l_alpha:
    aph = l_alpha[key]
    for i_alpha in range(0, len(aph)):            
        alpha = aph[i_alpha]
        model = pulp.LpProblem("First_Model", pulp.LpMaximize)

        weighted_sum(model, ofs_norn, [alpha, 0.5 * (1 - alpha), 0.5 * (1 - alpha)]) 

        const_idx = add_constraints(model, integer_vars, constraints, crops_year_integer, len(crops), YEARS, const_id = 0)
        const_idx = add_binary_correlation(model, binary_vars, integer_vars, 113, const_id = const_idx)
        const_idx = add_crop_rotation_constraints(model, integer_vars, crops_year_integer, rotations, YEARS, crops, 'y{:03d}_int_{}', const_idx)

        solution = model.solve(pulp.GLPK())

        integer_var_values = {key: pulp.value(value) for key, value in integer_vars.items()}
        binary_var_values = {key: pulp.value(value) for key, value in binary_vars.items()}

        ney_incone_ofs_values = sum_cost_of_values(income.loc[0,:], integer_var_values, crops_year_integer, 'net_income', 0, 4)
        us_ofs_values = sum_of_var_values(integer_var_values, 'ua', 0, 4)
        crop_diversity_ofs_values = sum_of_var_values(binary_var_values, 'crop_diversity', 0, 4)


        solution_lst.append([key,
                            i_alpha,
                            alpha,
                            str(pulp.LpStatus[solution])] +
                            list(integer_var_values.values())+
                            list(binary_var_values.values())+
                            [pulp.value(model.objective)] +
                            list(ney_incone_ofs_values.values())+
                            list(us_ofs_values.values())+
                            list(crop_diversity_ofs_values.values())
                            )


GLPSOL--GLPK LP/MIP Solver 5.0
Parameter(s) specified in the command line:
 --cpxlp /tmp/af8e6b432a794cc6903663d3dc03b2be-pulp.lp -o /tmp/af8e6b432a794cc6903663d3dc03b2be-pulp.sol
Reading problem data from '/tmp/af8e6b432a794cc6903663d3dc03b2be-pulp.lp'...
88 rows, 48 columns, 430 non-zeros
48 integer variables, 24 of which are binary
326 lines were read
GLPK Integer Optimizer 5.0
88 rows, 48 columns, 430 non-zeros
48 integer variables, 24 of which are binary
Preprocessing...
24 constraint coefficient(s) were reduced
64 rows, 48 columns, 242 non-zeros
48 integer variables, 24 of which are binary
Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  2.260e+03  ratio =  2.260e+03
GM: min|aij| =  4.453e-01  max|aij| =  2.246e+00  ratio =  5.042e+00
EQ: min|aij| =  2.100e-01  max|aij| =  1.000e+00  ratio =  4.761e+00
2N: min|aij| =  1.104e-01  max|aij| =  1.562e+00  ratio =  1.416e+01
Constructing initial basis...
Size of triangular part is 64
Solving LP relaxation...
GLPK Simplex Optimizer 5.

In [16]:
columns = ["alpha_type", "iter", "alpha", "status"] +\
         list(integer_var_values.keys()) + \
         list(binary_var_values.keys()) + \
         ['OF_Optimized'] + \
         list(ney_incone_ofs_values.keys()) + \
         list(us_ofs_values.keys()) + \
         list(crop_diversity_ofs_values.keys())

In [17]:
results = pd.DataFrame(solution_lst, columns=columns)

In [19]:
l = {'alpha': 'λ', 'OF_Optimized': 'g'}

fig = plot_one_scatter(results, 'alpha', 'OF_Optimized', l)

fig.show()
#fig.write_image('images/fig_mdl2_z2y_vs_lambda.svg')
#tikzplotly.save("figure.tex", fig)

In [20]:
l = {'alpha': 'λ', 'net_income_total': 'g<sub>1</sub> (USD)'}

fig = plot_one_scatter(results, 'alpha', 'net_income_total', l)

fig.show()
#fig.write_image('images/fig_mdl2_z2y_vs_lambda.svg')
#tikzplotly.save("figure.tex", fig)

In [21]:
cm = ['rgb(77,77,77)', 'rgb(153, 153, 153)', 'rgb(186,186,186)', 'rgb(244,165,130)', 'rgb(214,96,77)','rgb(230, 39, 23)']
cm = ['rgb(77,77,77)', 'rgb(153, 153, 153)', 'rgb(214,96,77)','rgb(230, 39, 23)']

In [22]:
'y001_int_Wheat', 'y001_int_Beetroot', 'y001_int_Cloves', 'y001_int_Broad_Bean', 'y001_int_Green_Onion', 'y001_int_Autumn_Potatoes', 
'y001_int_Sunflower', 'y001_int_Maize', 'y001_int_Tomatoes', 'y001_int_Cucumbers', 'y001_int_Okra', 'y001_int_Watermelon', 

'y002_int_Wheat', 'y002_int_Beetroot', 'y002_int_Cloves', 'y002_int_Broad_Bean', 'y002_int_Green_Onion', 'y002_int_Autumn_Potatoes', 
'y002_int_Sunflower', 'y002_int_Maize', 'y002_int_Tomatoes', 'y002_int_Cucumbers', 'y002_int_Okra', 'y002_int_Watermelon',

('y002_int_Sunflower',
 'y002_int_Maize',
 'y002_int_Tomatoes',
 'y002_int_Cucumbers',
 'y002_int_Okra',
 'y002_int_Watermelon')

In [23]:
l = {   'alpha':'λ', 
                                        'f1': 'g<sub>1</sub> (USD)',
                                        'X_1_1':'u<sub>1</sub>', 
                                        'X_1_2':'u<sub>2</sub>', 
                                        'X_1_3':'u<sub>3</sub>', 
                                        'X_1_4':'u<sub>4</sub>', 
                                        'X_1_5':'u<sub>5</sub>', 
                                        'X_1_6':'u<sub>6</sub>', 
                                        'X_1_7':'u<sub>7</sub>', 
                                        'X_1_8':'u<sub>8</sub>', 
                                        'X_1_9':'u<sub>9</sub>', 
                                        'X_1_10':'u<sub>10</sub>', 
                                        'X_1_11':'u<sub>11</sub>', 
                                        'X_1_12':'u<sub>12</sub>'}

In [24]:
px.data.iris()

,sepal_length,sepal_width,petal_length,petal_width,species,species_id
0,5.1,3.5,1.4,0.2,setosa,1
1,4.9,3.0,1.4,0.2,setosa,1
2,4.7,3.2,1.3,0.2,setosa,1
3,4.6,3.1,1.5,0.2,setosa,1
4,5.0,3.6,1.4,0.2,setosa,1
...,...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,virginica,3
146,6.3,2.5,5.0,1.9,virginica,3
147,6.5,3.0,5.2,2.0,virginica,3
148,6.2,3.4,5.4,2.3,virginica,3


In [61]:
dt = results
fig1 = px.parallel_coordinates(dt, 
                             dimensions=['alpha',
                                         'y001_int_Wheat', 'y001_int_Beetroot', 'y001_int_Cloves', 'y001_int_Broad_Bean', 'y001_int_Green_Onion', 'y001_int_Autumn_Potatoes', 
'y001_int_Sunflower', 'y001_int_Maize', 'y001_int_Tomatoes', 'y001_int_Cucumbers', 'y001_int_Okra', 'y001_int_Watermelon'],
                             color="net_income_total", 
                             color_continuous_scale=cm,
                             labels=None,
                             color_continuous_midpoint=2,
                             range_color = [dt.net_income_total.min(), dt.net_income_total.max()])

fig1.show()

In [62]:
tikzplotly.save("figure.tex", fig)

/home/roberto/anaconda3/envs/OtimizacaoLinear/lib/python3.10/site-packages/tikzplotly/_save.py:59: UserWarning:

Trace type splom is not supported yet.



In [60]:
_columns = ["alpha","net_income_total", "crop_diversity_total"]
fig = px.scatter_matrix(dt,
    dimensions= _columns,
    #color="species", symbol="species",
    #title="Scatter matrix of iris data set",
    labels={col:col.replace('_', ' ') for col in _columns}
    ) # remove underscore
fig.update_traces(diagonal_visible=False,
                  marker=dict(color='rgb(115, 115, 115)',
                              size=3)
                )

fig.update_layout(
    xaxis=dict(
        showline=True,
        showgrid=False,
        showticklabels=True,
        linecolor='rgb(204, 204, 204)',
        linewidth=2,
        ticks='outside',
        tickfont=dict(
            family='Arial',
            size=12,
            color='rgb(82, 82, 82)',
        ),
    ),
    yaxis=dict(
        showgrid=True,
        zeroline=False,
        showline=False,
        showticklabels=True,
        linecolor='rgb(204, 204, 204)',
        linewidth=2,
        ticks='outside',
        tickfont=dict(
            family='Arial',
            size=12,
            color='rgb(82, 82, 82)',
        ),
    ),

    showlegend=True,
    autosize=True,
    #plot_bgcolor='#000000'
)

fig.show()